In [1]:
# Import standard library modules
import sys

# Set the relative path to the project root directory
relative_path_to_root = "../../../"

# Add the project root to the system path for importing in-house modules
sys.path.append(relative_path_to_root)

# Import in-house modules from the 'utilities' package
from utilities import load_data
from utilities import temporal_train_test_split
from utilities import print_title, print_label

In [2]:
# Data manipulation and analysis
import pandas as pd

# Date and time manipulation
from datetime import date

# File and directory manipulation
from pathlib import Path

# Data preprocessing and model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

# Model evaluation metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import numpy as np

# Machine learning models (regression)
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor

# Ensemble methods
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [1]:
def split_dataset_by_date(raw_data: pd.DataFrame, todays_date: str):
    # Filter data by today's date
    filter_data_by_date = raw_data["Date"] == todays_date
    
    # Create a new dataframe with today's data
    todays_data = raw_data[filter_data_by_date].reset_index(drop=True)
    
    # Create a new dataframe with historical data (excluding today's data)
    historical_data = raw_data[~filter_data_by_date].reset_index(drop=True)
    
    return historical_data, todays_data

NameError: name 'pd' is not defined

In [4]:
file_name = "sp500_adj_close_raw_with_nas"
file_path = f"../../../data/raw_data/{file_name}"

raw_data = load_data(file_path)

╔═══════════════════════════════════════════════════════════════╗
║File `test_w_na.csv.bz2` loaded from `sp500_adj_close_raw_with_nas.zip`║
╚═══════════════════════════════════════════════════════════════╝


### Data Pre-Processing
___

#### Split todays data (For prediction) and historical data (For training)

In [5]:
todays_date = "2024-10-25"

historical_data, todays_data = split_dataset_by_date(raw_data, todays_date)

print("Todays Date:", todays_date)

Todays Date: 2024-10-25


#### Ensure there are missing values on Todays Data is what we are predicting...

In [6]:
todays_data.isnull().sum()

Date                    0
Ticker                  0
Adjusted Close          0
Next Day Close        501
Previous Day Close      0
Return                  0
Volatility              0
RSI                     0
SMA_50                  0
SMA_100                 0
SMA_200                 0
Upper Band              0
Lower Band              0
Support                 0
Resistance              0
Action                501
dtype: int64

#### Handle missing values (NA's) on the historical data used to train and test...

In [7]:
def clean_historical_data(historical_data: pd.DataFrame) -> pd.DataFrame:
    # Remove all rows where `Adjusted Close` is 0
    historical_data = historical_data[historical_data["Adjusted Close"] != 0].reset_index(drop=True)
    
    # Remove all rows where `Volatility` is NaN
    historical_data = historical_data.dropna(subset=["Volatility"]).reset_index(drop=True)
    
    # Backfill the `RSI` column
    historical_data["RSI"] = historical_data["RSI"].bfill()
    
    # Backfill the `Action` column
    historical_data["Action"] = historical_data["Action"].bfill()
    
    return historical_data

historical_data = clean_historical_data(historical_data)

historical_data.isnull().sum()

Date                  0
Ticker                0
Adjusted Close        0
Next Day Close        0
Previous Day Close    0
Return                0
Volatility            0
RSI                   0
SMA_50                0
SMA_100               0
SMA_200               0
Upper Band            0
Lower Band            0
Support               0
Resistance            0
Action                0
dtype: int64

### Exploratory Data Analysis (EDA):
___

#### Todays Data:

In [8]:
print("Shape:", todays_data.shape)
display(todays_data.head(2))
display(todays_data.tail(2))

Shape: (501, 16)


,Date,Ticker,Adjusted Close,Next Day Close,Previous Day Close,Return,Volatility,RSI,SMA_50,SMA_100,SMA_200,Upper Band,Lower Band,Support,Resistance,Action
0,2024-10-25,A,130.16,NaN,130.69,-0.004055,0.012137,18.866226,140.33243,136.65535,137.94844,152.15059,129.75082,130.16,148.244,NaN
1,2024-10-25,AAPL,231.40,NaN,230.57,0.003600,0.013669,66.374374,226.80140,221.79668,200.76884,237.80908,222.31091,216.32,236.480,NaN


,Date,Ticker,Adjusted Close,Next Day Close,Previous Day Close,Return,Volatility,RSI,SMA_50,SMA_100,SMA_200,Upper Band,Lower Band,Support,Resistance,Action
499,2024-10-25,ZBRA,359.97,NaN,362.05,-0.005745,0.009788,43.769238,355.8908,336.8905,311.04135,380.01144,359.82355,320.77,377.68,NaN
500,2024-10-25,ZTS,179.91,NaN,181.50,-0.008760,0.012576,36.496883,189.0940,183.3149,179.24370,197.88783,182.27017,179.91,196.48,NaN


#### Historical Data:

In [9]:
print("Shape:", historical_data.shape)
display(historical_data.head(2))
display(historical_data.tail(2))

Shape: (1978479, 16)


,Date,Ticker,Adjusted Close,Next Day Close,Previous Day Close,Return,Volatility,RSI,SMA_50,SMA_100,SMA_200,Upper Band,Lower Band,Support,Resistance,Action
0,2008-01-02,A,23.256388,23.025743,23.538280,-0.011976,0.015704,48.82759,23.314175,23.299887,23.564934,24.727251,22.540236,21.392029,24.351946,short
1,2008-01-02,AAPL,5.876342,5.879055,5.974057,-0.016357,0.018937,59.06735,5.518483,4.939064,4.197630,6.135833,5.403559,4.637376,6.026840,buy


,Date,Ticker,Adjusted Close,Next Day Close,Previous Day Close,Return,Volatility,RSI,SMA_50,SMA_100,SMA_200,Upper Band,Lower Band,Support,Resistance,Action
1978477,2024-10-24,ZBRA,362.05,359.97,368.09,-0.016409,0.010532,44.39913,355.5556,336.31850,310.51785,379.32880,361.36920,320.77,377.68,short
1978478,2024-10-24,ZTS,181.50,179.91,188.99,-0.039632,0.013328,35.00878,189.1774,183.23051,179.33058,197.27248,184.37552,180.90,196.48,short


### Create Multiple Versions of Dataset
___

#### Select which version of the data to work with

In [10]:
main_data = historical_data.copy().reset_index(drop=True)

# Create multiple versions of the dataset

# Data with out dates and tickers (Set as index for reference)
data_v1 = main_data.copy().set_index(["Date", "Ticker"])

# Data with dates seperated into year, month, and day
data_v2 = main_data.copy()
data_v2["Date"] = pd.to_datetime(data_v2["Date"])
data_v2["Year"] = data_v2["Date"].dt.year
data_v2["Month"] = data_v2["Date"].dt.month
data_v2["Day"] = data_v2["Date"].dt.day
data_v2 = data_v2.set_index(["Date", "Ticker"])

##### **Note: It is recommended to remove `["Previous Day Close", "Resistance", "Upper Band", "SMA_50", "SMA_200"]` after VIF inspection...**

##### **Note: It is recommended to remove `["Day"]` after p-value inspection...**

In [11]:
select_data = data_v2.copy()

select_columns_to_drop = ["Action", "Previous Day Close", "SMA_50", "Resistance", "Upper Band", "SMA_200", "Day"]

data = select_data.drop(columns=select_columns_to_drop)

print("Shape:", data.shape)
data.head()

Shape: (1978479, 10)


Adjusted Close  Next Day Close    Return  Volatility  \
Date       Ticker                                                         
2008-01-02 A            23.256388       23.025743 -0.011976    0.015704   
           AAPL          5.876342        5.879055 -0.016357    0.018937   
           ABT          18.130209       18.019754 -0.006055    0.010484   
           ACGL          7.608889        7.764444 -0.026581    0.016022   
           ACN          26.437077       25.982521  0.000833    0.024039   

                         RSI    SMA_100  Lower Band    Support  Year  Month  
Date       Ticker                                                            
2008-01-02 A       48.827590  23.299887   22.540236  21.392029  2008      1  
           AAPL    59.067350   4.939064    5.403559   4.637376  2008      1  
           ABT     34.677372  17.628250   18.221804  16.775562  2008      1  
           ACGL    45.154190   7.878933    7.378535   7.463333  2008      1  
           ACN     54.812176  27.784420   24.273775  24.765512  2008      1

### Split data features `X` and target `y`
___

In [12]:
# Split the data into features (X) and target (y)
X = data.drop(columns="Next Day Close")

y = data["Next Day Close"]

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

Shape of X: (1978479, 9)
Shape of y: (1978479,)


### Data Splitting
___

In [13]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2, # 80% training and 20% testing
    random_state=42
)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (1582783, 9)
Shape of X_test: (395696, 9)
Shape of y_train: (1582783,)
Shape of y_test: (395696,)


### Scaling the Data
___

#### Scale the data using `StandardScaler`

In [16]:
# Scale using StandardScaler
X_scaler = StandardScaler()
y_scaler = StandardScaler()

X_train_scaled = X_scaler.fit_transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = y_scaler.transform(y_test.values.reshape(-1, 1))

print("X_train_scaled shape:", X_train_scaled.shape)
print("X_test_scaled shape:", X_test_scaled.shape)
print("y_train_scaled shape:", y_train_scaled.shape)
print("y_test_scaled shape:", y_test_scaled.shape)

X_train_scaled shape: (1582783, 9)
X_test_scaled shape: (395696, 9)
y_train_scaled shape: (1582783, 1)
y_test_scaled shape: (395696, 1)


### Model Training
___


In [19]:
y_train_scaled.reshape(-1).shape

(1582783,)

In [22]:
ran_for = RandomForestRegressor(
    n_estimators=50,
    max_depth=10,
    max_features="sqrt",
    random_state=42,
    n_jobs=-1
)

ran_for.fit(X_train_scaled, y_train_scaled.reshape(-1))

RandomForestRegressor(max_depth=10, max_features='sqrt', n_estimators=50,
                      n_jobs=-1, random_state=42)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "n_estimators": [50, 100, 150],
    "max_depth": [5, 10, 15],
    "max_features": ["auto", "sqrt", "log2"]
}

grid_search = GridSearchCV(
    estimator=ran_for,
    param_grid=param_grid,
    cv=5,
    n_jobs=-1
)

grid_search.fit(X_train_scaled, y_train_scaled.reshape(-1))

In [ ]:
print(

### Model Evaluation
___

In [23]:
def adj_r2_score(model, X, y):
    r2 = model.score(X, y)
    n_cols = X.shape[1]
    return 1 - (1 - r2) * (len(y) - 1) / (len(y) - n_cols - 1)

In [42]:
y_predict_scaled = ran_for.predict(X_test_scaled)

y_predict_unscaled = y_scaler.inverse_transform(y_predict_scaled.reshape(-1, 1))
y_test_unscaled = y_scaler.inverse_transform(y_test_scaled.reshape(-1, 1))

mse = mean_squared_error(y_test_unscaled, y_predict_unscaled)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_unscaled, y_predict_unscaled)
adj_r2 = adj_r2_score(ran_for, X_test_scaled, y_test_scaled)

print_title("Random Forest Regressor", closed_corners=False)
print_label("Mean Squared Error:", mse)
print_label("Root Mean Squared Error:", rmse)
print_label("R-Squared (Training Data):", ran_for.score(X_train_scaled, y_train_scaled))
print_label("R-Squared (Testing Data):", r2)
print_label("Adjusted R-Squared:", adj_r2, closed_corners=True) 

╔═══════════════════════════════════════════════════════════════╗
║                    Random Forest Regressor                    ║
╠═══════════════════════════════════════════════════════════════╣
║ Mean Squared Error:            |           20.15681           ║
║ Root Mean Squared Error:       |           4.48963            ║
║ R-Squared (Training Data):     |           0.99965            ║
║ R-Squared (Testing Data):      |           0.99954            ║
║ Adjusted R-Squared:            |           0.99954            ║
╚═══════════════════════════════════════════════════════════════╝


In [ ]:
cv_scores = cross_val_score(
    LinearRegression(
        n_jobs=-1
    ),
    X_train_scaled,
    y_train_scaled,
    scoring="r2",
    cv=10
)

print_title("Cross Validation Scores", closed_corners=False)

for index, score in enumerate(cv_scores):
    print_label(f"Fold {index+1}:", score)

print_label("", "")
print_label("Mean R^2 Score:", cv_scores.mean())
print_label("Standard Deviation:", cv_scores.std(), closed_corners=True)

### Predict tomorrows `Adjusted Close`
___

In [35]:
def preprocess_todays_data(df, columns_to_drop):
    # Convert 'Date' column to datetime
    df["Date"] = pd.to_datetime(df["Date"])
    
    # Extract year, month, and day from 'Date' column
    df["Year"] = df["Date"].dt.year
    df["Month"] = df["Date"].dt.month
    df["Day"] = df["Date"].dt.day
    
    # Drop specified columns
    df = df.drop(columns=columns_to_drop)
    
    # Set 'Date' and 'Ticker' as the index
    df = df.set_index(["Date", "Ticker"])
    
    # Drop 'Next Day Close' column
    df = df.drop(columns="Next Day Close")
    
    return df

X_to_predict = todays_data.copy()

X_to_predict = preprocess_todays_data(X_to_predict, select_columns_to_drop)

X_to_predict_scaled = X_scaler.transform(X_to_predict)

print("Shape of todays_data_clean_scaled:", X_to_predict_scaled.shape)
X_to_predict_scaled[:5]

Shape of todays_data_clean_scaled: (501, 9)


array([[ 0.21029221, -0.21050541, -0.50676969, -2.04849203,  0.26539134,
         0.25211298,  0.28301857,  1.62443221,  1.02782802],
       [ 0.71656911,  0.1299215 , -0.38651463,  0.79365625,  0.70975971,
         0.74646253,  0.76314191,  1.62443221,  1.02782802],
       [ 0.49863601, -0.4592691 , -0.79119435, -0.85020721,  0.50742677,
         0.5520643 ,  0.59719398,  1.62443221,  1.02782802],
       [ 0.23249558,  0.58958269, -0.28334676,  0.37484342,  0.24850492,
         0.22527628,  0.19452787,  1.62443221,  1.02782802],
       [ 0.13068012, -0.90781959, -0.64725082,  0.08741691,  0.12313012,
         0.15204863,  0.17207307,  1.62443221,  1.02782802]])

In [37]:
y_to_predict_scaled = ran_for.predict(X_to_predict_scaled)

y_to_predict_unscaled = y_scaler.inverse_transform(y_to_predict_scaled.reshape(-1, 1))

print("Shape of y_to_predict_unscaled:", y_to_predict_unscaled.shape)
y_to_predict_unscaled[:5]

Shape of y_to_predict_unscaled: (501, 1)


array([[133.24775894],
       [236.21700398],
       [190.69137901],
       [133.94288166],
       [115.00592078]])

In [38]:
prediction_df = todays_data.copy()

prediction_df["Next Day Close"] = y_to_predict_unscaled.round(3)

prediction_df = prediction_df[["Date", "Ticker", "Adjusted Close", "Next Day Close"]]

prediction_df = prediction_df.set_index(["Date", "Ticker"])

print("Shape:", prediction_df.shape)
display(prediction_df.head(20))
display(prediction_df.tail(20))

Shape: (501, 2)


Adjusted Close  Next Day Close
Date       Ticker                                
2024-10-25 A              130.160         133.248
           AAPL           231.400         236.217
           ABBV           187.820         190.691
           ABNB           134.600         133.943
           ABT            114.240         115.006
           ACGL           105.280         107.010
           ACN            360.790         363.114
           ADBE           483.730         490.960
           ADI            230.170         231.499
           ADM             56.570          57.286
           ADP            287.960         291.086
           ADSK           286.050         286.243
           AEE             87.800          88.128
           AEP             98.180          99.740
           AES             16.965          17.093
           AFL            109.710         111.464
           AIG             76.820          77.103
           AIZ            191.780         196.357
           AJG            283.280         285.620
           AKAM           101.980         101.577

Adjusted Close  Next Day Close
Date       Ticker                                
2024-10-25 WBD              7.590           8.165
           WDC             69.455          69.248
           WEC             97.170          97.432
           WELL           130.190         130.418
           WFC             64.520          62.804
           WM             206.800         209.566
           WMB             52.510          51.569
           WMT             82.510          82.606
           WRB             58.170          59.284
           WST            307.920         306.930
           WTW            290.410         294.811
           WY              31.785          32.205
           WYNN            97.980          97.730
           XEL             64.460          64.374
           XOM            119.510         119.626
           XYL            130.410         132.453
           YUM            133.040         134.965
           ZBH            102.330         104.541
           ZBRA           359.970         363.588
           ZTS            179.910         184.014